In [24]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

tf.__version__, hub.__version__

('2.0.0-beta1', '0.5.0')

In [11]:
df = pd.read_csv('../data/human_label.csv')

In [12]:
df.rename(index=str, columns={"Human Label": "label"}, inplace=True)
df.rename(index=str, columns={"cleansed_comment": "text"}, inplace=True)

In [13]:
df = df[df.label != 'Neutral'] # filter out Neutral, it is not in our model
df.label = df.label.apply(lambda x: 1 if x == 'Positive' else 0)
df.reset_index(drop=True, inplace=True)

In [7]:
all_models = {}
all_models['gnews_swivel'] = tf.keras.models.load_model('../models/gnews_swivel_1_ep_model.h5',
                                                        custom_objects={ 'KerasLayer': hub.KerasLayer })
all_models['nnlm_en_50'] = tf.keras.models.load_model('../models/nnlm_en_50_1_ep_model.h5',
                                                      custom_objects={ 'KerasLayer': hub.KerasLayer })
all_models['nnlm_en_128'] = tf.keras.models.load_model('../models/nnlm_en_128_1_ep_model.h5',
                                                       custom_objects={ 'KerasLayer': hub.KerasLayer })

In [9]:
metrics = {
    'model_name': [],
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1_score': [],
}

In [25]:
for name, model in all_models.items():
    yhat_probs = model.predict(df.text.values)[:, 0]
    yhat_classes = model.predict_classes(df.text.values)[:, 0]
    accuracy = accuracy_score(df.label.values, yhat_classes)
    precision = precision_score(df.label.values, yhat_classes)
    recall = recall_score(df.label.values, yhat_classes)
    f1 = f1_score(df.label.values, yhat_classes)
    
    metrics['model_name'].append(name)
    metrics['accuracy'].append(accuracy)
    metrics['precision'].append(precision)
    metrics['recall'].append(recall)
    metrics['f1_score'].append(f1)

In [27]:
met = pd.DataFrame.from_dict(metrics)
met

,model_name,accuracy,precision,recall,f1_score
0,gnews_swivel,0.929351,0.928251,0.841463,0.882729
1,nnlm_en_50,0.956969,0.956989,0.904472,0.929990
2,nnlm_en_128,0.959538,0.938650,0.932927,0.935780
